# Step 1: Stimulate Rankings of Relevance for E and P (5 points)

In [1]:
import itertools as iter
import numpy as np
import random

#The gradings
relevance_gradings = ['N', 'R', 'HR']

# Create all possible versions of length five rankings
list_p = [p for p in iter.product(relevance_gradings, repeat=5)]
list_e = [p for p in iter.product(relevance_gradings, repeat=5)]

print(len(list_e))

# Create all possible pairs of the two lists - if computing power permits
#list_pairs = [list(zip(list_p, p)) for p in itertools.permutations(list_e)]

# Sampling - take one at random from list 1 and one 1 from list 2 until 1000.
sample_size = 1000
sample_list_p = [random.choice(list_p) for _ in range(sample_size)]
sample_list_e = [random.choice(list_e) for _ in range(sample_size)]

sample_pairs = list(zip(sample_list_p, sample_list_e))

for i in range(0,5):
    print(sample_pairs[i])

243
(('R', 'HR', 'R', 'HR', 'HR'), ('R', 'R', 'N', 'N', 'HR'))
(('HR', 'R', 'R', 'HR', 'R'), ('N', 'N', 'R', 'R', 'N'))
(('N', 'R', 'R', 'R', 'R'), ('R', 'HR', 'R', 'N', 'HR'))
(('N', 'N', 'N', 'HR', 'N'), ('R', 'R', 'R', 'HR', 'N'))
(('HR', 'HR', 'N', 'N', 'R'), ('N', 'N', 'R', 'N', 'R'))


In [2]:
# Binary - Precision at rank K
def calc_ave_precision(list_of_relevance):
    rank_length = len(list_of_relevance)
    running_score = 0
    relevant_count = 0
    for i in range(0, rank_length):
        if(list_of_relevance[i] == "R" or list_of_relevance[i] == "HR"):
            relevant_count += 1
            running_score += relevant_count / (i+1)
    ave_precision = running_score / rank_length
    return ave_precision

test_p = sample_pairs[0][0]
test_e = sample_pairs[0][1]

print(test_p)
print(calc_ave_precision(test_p))
print(test_e)
print(calc_ave_precision(test_e))

#

('R', 'HR', 'R', 'HR', 'HR')
0
1.0
2.0
3.0
4.0
1.0
('R', 'R', 'N', 'N', 'HR')
0
1.0
2.0
2.0
2.0
0.52
